In [4]:
from transformer_lens import HookedTransformer
import transformer_lens
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch



In [5]:
devices = transformer_lens.utilities.devices.AvailableDeviceMemory

In [6]:
devices

list[tuple[int, int]]

In [ ]:
reference_model_path = 'meta-llama/Llama-3.1-8B'
baseline_model_path = 'meta-llama/Llama-3.1-8B'

In [ ]:
baseline_model_device='mps'

In [ ]:
baseline_model_hf = AutoModelForCausalLM.from_pretrained(baseline_model_path, torch_dtype=torch.bfloat16)
baseline_model_tokenizer = AutoTokenizer.from_pretrained(baseline_model_path)

baseline_model = HookedTransformer.from_pretrained_no_processing(
    reference_model_path,
    hf_model=baseline_model_hf,
    tokenizer=baseline_model_tokenizer,
    device=baseline_model_device,
    move_to_device=True,
)

